# **📓 00: Preparación y Auditoría de Datos**

Este notebook es el centro de control para la gestión de la calidad de los datos del proyecto VIREC. Contiene tres herramientas principales:

1.  **Sanitizador de Datos Crudos:** Convierte formatos de imagen (`.heic` a `.jpg`), elimina duplicados reales basándose en el contenido (hash) y renombra archivos con nombres conflictivos.
2.  **Auditoría del Pipeline Completo:** Compara la "fuente de la verdad" (la hoja de etiquetas) con los archivos físicos en todas las carpetas del proyecto para encontrar inconsistencias como duplicados, archivos fantasma o archivos huérfanos.
3.  **Herramienta de Reseteo:** Permite mover todos los archivos procesados de vuelta a las carpetas de datos crudos para realizar una resincronización completa, ideal para cuando se cambian los parámetros de división del dataset.

**Instrucciones:** Ejecuta las celdas de "Setup" y "Configuración" primero. Luego, ejecuta la celda correspondiente a la tarea que deseas realizar.

In [ ]:
# ====================================================================================
# @title PASO 1: SETUP DEL ENTORNO
# ====================================================================================
# Instalar librerías para conversión de HEIC y manejo de Hojas de Cálculo
!pip install -r requirements.txt -q

import pandas as pd
import os
import shutil
import re
import hashlib
from collections import defaultdict
from PIL import Image
import pillow_heif
from google.colab import auth, drive
import gspread
from google.auth import default

# --- Montar Drive y Autenticar para Google Sheets ---
if not os.path.exists('/content/drive/MyDrive'):
    drive.mount('/content/drive')
else:
    print("Google Drive ya está montado.")

try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    print("✅ Autenticación con Google Sheets exitosa.")
except Exception as e:
    print(f"⚠️ Advertencia: No se pudo autenticar con Google Sheets. El registro de resultados podría fallar. Error: {e}")

print("✅ Librerías instaladas.")

### **Configuración del Entorno y Búsqueda de Rutas**
Esta celda monta Google Drive y busca automáticamente la carpeta raíz del proyecto (`Proyecto_VIREC`). A partir de esta ruta base, construye todas las demás rutas de directorios que se utilizarán en las siguientes herramientas. Esto asegura que el notebook funcione para cualquier miembro del equipo sin necesidad de modificar las rutas manualmente.

In [ ]:
# ====================================================================================
# @title PASO 2: CONFIGURACIÓN DE RUTAS Y CONEXIÓN
# ====================================================================================

# Montar Google Drive
if not os.path.exists('/content/drive/MyDrive'):
    drive.mount('/content/drive')
else:
    print("Google Drive ya está montado.")

# --- Búsqueda Dinámica de la Ruta Base del Proyecto ---
NOMBRE_CARPETA_ANCLA = 'Proyecto_VIREC'
RUTA_BASE_PROYECTO = None
print(f"\nBuscando la carpeta ancla '{NOMBRE_CARPETA_ANCLA}'...")
for root, dirs, files in os.walk('/content/drive/MyDrive'):
    if NOMBRE_CARPETA_ANCLA in dirs:
        RUTA_BASE_PROYECTO = os.path.join(root, NOMBRE_CARPETA_ANCLA)
        break

if not RUTA_BASE_PROYECTO:
    raise FileNotFoundError(f"❌ ERROR CRÍTICO: No se pudo encontrar la carpeta '{NOMBRE_CARPETA_ANCLA}'. Asegúrate de que tenga un acceso directo en tu 'Mi unidad'.")
else:
    print(f"✅ ¡Proyecto encontrado! La ruta base es: {RUTA_BASE_PROYECTO}")
    
    # --- Construcción de todas las rutas del proyecto ---
    RUTA_DATASET = os.path.join(RUTA_BASE_PROYECTO, 'dataset')
    RUTA_FOTOS_CRUDAS_PROPIAS = os.path.join(RUTA_DATASET, 'fotos_crudas_propias')
    RUTA_FOTOS_CRUDAS_EXTERNAS = os.path.join(RUTA_DATASET, 'fotos_crudas_externas')
    RUTA_DATASET_FINAL_PROPIO = os.path.join(RUTA_DATASET, 'dataset_final_propio')
    RUTA_DATASET_FINAL_EXTERNO = os.path.join(RUTA_DATASET, 'dataset_final_externo')
    
    #TODO: Este nombre ahora debería venir de config.py
    NOMBRE_HOJA_ETIQUETAS = "VIREC - Hoja de Etiquetas del Dataset"
    
    print("✅ Rutas configuradas exitosamente.")

### **Herramienta 1: Auditoría Completa del Pipeline**
Esta herramienta realiza una "radiografía" completa de tu proyecto. Compara el registro central de etiquetas (el archivo `.csv` exportado) con todos los archivos de imagen físicos que existen en las carpetas de datos (`fotos_crudas` y `dataset_final`).

El informe final te alertará sobre tres tipos de posibles inconsistencias:
*   **Duplicados:** Nombres de archivo que aparecen más de una vez en tu hoja de etiquetas.
*   **Fantasmas:** Entradas en tu hoja de etiquetas que no corresponden a ningún archivo de imagen real.
*   **Huérfanos:** Archivos de imagen en tus carpetas que no están registrados en la hoja de etiquetas.

In [ ]:
# ====================================================================================
# @title PASO 3: EJECUTAR AUDITORÍA COMPLETA
# ====================================================================================

# --- 1. Recolectar todos los archivos físicos del proyecto ---
todos_los_archivos_fisicos = set()
carpetas_a_escanear = [
    RUTA_FOTOS_CRUDAS_PROPIAS,
    RUTA_FOTOS_CRUDAS_EXTERNAS,
    RUTA_DATASET_FINAL_PROPIO,
    RUTA_DATASET_FINAL_EXTERNO
]
print("\nIniciando escaneo de todas las carpetas de datos...")
for carpeta_base in carpetas_a_escanear:
    if os.path.exists(carpeta_base):
        print(f"  Escanenado: {os.path.relpath(carpeta_base, RUTA_BASE_PROYECTO)}...")
        for root, dirs, files in os.walk(carpeta_base):
            # Ignorar nuestras carpetas de control
            if '_convertidos' in dirs: dirs.remove('_convertidos')
            if '_duplicados_reales' in dirs: dirs.remove('_duplicados_reales')
            
            for filename in files:
                todos_los_archivos_fisicos.add(filename)
print(f"\nSe encontraron {len(todos_los_archivos_fisicos)} archivos físicos únicos en todo el proyecto.")

# --- 2. Conectarse a Google Sheets y realizar la auditoría ---
try:
    hoja_etiquetas = gc.open(NOMBRE_HOJA_ETIQUETAS).worksheet("Lista de etiquetas")
    
    # Leemos la hoja directamente a un DataFrame de pandas
    valores_crudos = hoja_etiquetas.get_all_values()
    df_etiquetas = pd.DataFrame(valores_crudos[1:], columns=valores_crudos[0])
    
    df_etiquetas.dropna(subset=['nombre_archivo'], inplace=True)
    print(f"Se leyeron {len(df_etiquetas)} registros directamente desde Google Sheets.")

    # Encontrar duplicados, fantasmas y huérfanos
    duplicados = df_etiquetas[df_etiquetas.duplicated(subset=['nombre_archivo'], keep=False)]
    csv_filenames_set = set(df_etiquetas['nombre_archivo'])
    ghost_files = sorted(list(csv_filenames_set - todos_los_archivos_fisicos))
    orphan_files = sorted(list(todos_los_archivos_fisicos - csv_filenames_set))

    # --- 3. Presentar el informe de auditoría ---
    print("\n" + "="*70)
    print("          INFORME DE AUDITORÍA COMPLETA DEL PIPELINE")
    print("="*70)
    
    # Reportar Duplicados
    if not duplicados.empty:
        print(f"\n❌ DUPLICADOS: Se encontraron {len(duplicados['nombre_archivo'].unique())} nombres de archivo repetidos en la hoja de etiquetas.")
        print("   ACCIÓN: Elimina las filas duplicadas en Google Sheets.\n")
        print(duplicados.sort_values(by='nombre_archivo').to_string())
    else:
        print("\n✅ DUPLICADOS: No se encontraron registros duplicados en la hoja de etiquetas.")

    # Reportar Fantasmas
    if ghost_files:
        print(f"\n❌ FANTASMAS: Se encontraron {len(ghost_files)} registros en la hoja que no tienen un archivo físico.")
        print("   ACCIÓN: Elimina estas filas de Google Sheets, ya que sus archivos no existen.\n")
        for i, filename in enumerate(ghost_files, 1):
            print(f"   {i}. {filename}")
    else:
        print("\n✅ FANTASMAS: No se encontraron registros sin archivo físico.")
        
    # Reportar Huérfanos
    if orphan_files:
        print(f"\n❌ HUÉRFANOS: Se encontraron {len(orphan_files)} archivos en disco que no están registrados en la hoja.")
        print("   ACCIÓN: Añade estos archivos a la hoja usando la herramienta 'Sincronizar' de Apps Script.\n")
        for i, filename in enumerate(orphan_files, 1):
            print(f"   {i}. {filename}")
    else:
        print("\n✅ HUÉRFANOS: No se encontraron archivos físicos sin registrar.")

    print("\n" + "="*70)
    print("                       RESUMEN FINAL")
    print("="*70)
    print(f"Registros totales leídos del CSV: {len(df_etiquetas)}")
    print(f"Nombres de archivo ÚNICOS en el CSV: {len(csv_filenames_set)}")
    print(f"Archivos físicos ÚNICOS en el disco: {len(todos_los_archivos_fisicos)}")
    print("="*70)

except gspread.exceptions.SpreadsheetNotFound:
    print(f"❌ ERROR: No se encontró la Hoja de Cálculo '{NOMBRE_HOJA_ETIQUETAS}'. Verifica el nombre en tu config.py.")
except Exception as e:
    print(f"❌ Ocurrió un error al leer la Hoja de Cálculo: {e}")

### **Herramienta 2: Reseteo Completo del Pipeline**
Esta herramienta es tu "botón de reinicio". Su función es deshacer el trabajo del script `01_Preprocesamiento`, moviendo todos los archivos de las carpetas `dataset_final` de vuelta a sus carpetas `fotos_crudas` correspondientes.

**¿Cuándo usar esto?**
*   Después de haber hecho cambios significativos en la hoja de etiquetas (como corregir muchas etiquetas) y querer reaplicar la división a todo el dataset.
*   Si quieres cambiar los porcentajes de la división `train/validation/test` en el script `01_Preprocesamiento`.

Al ejecutar esta celda, también se **eliminará el archivo de registro `dataset_map.csv`**, forzando al script de preprocesamiento a tratar todos los archivos como nuevos en su próxima ejecución.

In [ ]:
# ====================================================================================
# @title PASO 4: EJECUTAR RESETEO COMPLETO DEL PIPELINE
# ====================================================================================
import shutil

# --- 1. Función reutilizable para mover archivos de vuelta ---
def mover_archivos_a_crudo(ruta_dataset_final, ruta_fotos_crudas, nombre_fuente):
    """Mueve todos los archivos de una estructura de dataset final a su carpeta cruda."""
    print(f"\nIniciando reseteo para la fuente: '{nombre_fuente}'...")
    archivos_movidos = 0
    if not os.path.exists(ruta_dataset_final):
        print(f"La carpeta '{os.path.basename(ruta_dataset_final)}' no existe. No hay archivos que mover.")
        return 0

    os.makedirs(ruta_fotos_crudas, exist_ok=True)
    print(f"Buscando archivos en: {os.path.relpath(ruta_dataset_final, RUTA_BASE_PROYECTO)}")

    for root, dirs, files in os.walk(ruta_dataset_final):
        for filename in files:
            ruta_origen = os.path.join(root, filename)
            ruta_destino = os.path.join(ruta_fotos_crudas, filename)
            try:
                shutil.move(ruta_origen, ruta_destino)
                archivos_movidos += 1
            except Exception as e:
                print(f"  ❌ Error al mover {filename}: {e}")

    print(f" -> Se movieron {archivos_movidos} archivos de vuelta a '{os.path.basename(ruta_fotos_crudas)}'.")
    return archivos_movidos

# --- 2. Ejecutar el proceso de reseteo ---
total_movidos = 0
if 'RUTA_BASE_PROYECTO' in locals(): # Verificación de seguridad
    # --- ¡IMPORTANTE! Ubicación del archivo de registro ---
    # Lo definimos aquí porque este script es el único que lo borra
    RUTA_REGISTRO = os.path.join(RUTA_BASE_PROYECTO, 'dataset_map.csv')

    # Mover archivos de ambas fuentes
    total_movidos += mover_archivos_a_crudo(RUTA_DATASET_FINAL_PROPIO, RUTA_FOTOS_CRUDAS_PROPIAS, 'propio')
    total_movidos += mover_archivos_a_crudo(RUTA_DATASET_FINAL_EXTERNO, RUTA_FOTOS_CRUDAS_EXTERNAS, 'externo')

    # Eliminar el archivo de registro
    print("\nEliminando el archivo de registro `dataset_map.csv`...")
    if os.path.exists(RUTA_REGISTRO):
        os.remove(RUTA_REGISTRO)
        print("✅ Archivo 'dataset_map.csv' eliminado exitosamente.")
    else:
        print("El archivo 'dataset_map.csv' no existía. No se hizo nada.")

    # --- 3. Informe final ---
    print("\n" + "="*60)
    print("              PROCESO DE RESETEO COMPLETO")
    print("="*60)
    print(f"Se movieron un total de {total_movidos} archivos de vuelta a sus carpetas crudas.")
    print("El archivo de registro `dataset_map.csv` ha sido eliminado.")
    print("\n✅ El entorno está listo para una resincronización limpia desde cero.")
    print("   Ahora puedes ejecutar el script `01_Preprocesamiento_Dataset.ipynb`.")
    print("="*60)
else:
    print("❌ ERROR: La variable 'RUTA_BASE_PROYECTO' no está definida. Ejecuta la celda de configuración (Paso 2) primero.")

### **Herramienta 3: Sanitización de Datos Crudos**
Esta es la herramienta de preparación de datos más importante. Su función es tomar las carpetas de datos crudos (`fotos_crudas_propias` y `fotos_crudas_externas`) y limpiarlas para que solo contengan archivos válidos, únicos y listos para ser etiquetados.

**Ejecuta esta celda DESPUÉS de que tu equipo haya subido un nuevo lote de fotos y ANTES de usar la herramienta "Sincronizar" en Google Sheets.**

El proceso de sanitización realiza 3 fases en cada carpeta:
1.  **Conversión de HEIC:** Convierte todos los archivos `.heic` a formato `.jpg` y mueve los originales a una subcarpeta `_convertidos`.
2.  **Gestión de Duplicados Reales:** Calcula la "huella digital" (hash) de cada imagen. Si encuentra archivos con contenido idéntico, conserva uno y mueve los demás a `_duplicados_reales`.
3.  **Resolución de Colisiones de Nombres:** Si encuentra archivos con contenido diferente pero nombres conflictivos (ej. `foto.jpg` y `foto (1).jpg`), los renombra automáticamente a un formato único (ej. `foto_1.jpg` y `foto_2.jpg`) para evitar ambigüedades.

In [ ]:
# ====================================================================================
# @title PASO 5: EJECUTAR SANITIZACIÓN DE DATOS CRUDOS
# ====================================================================================
import re
import hashlib
from PIL import Image
import pillow_heif
from collections import defaultdict

# --- 1. Funciones de procesamiento avanzado ---
def calcular_hash(ruta_archivo):
    """Calcula el hash SHA256 de un archivo para identificar su contenido único."""
    sha256_hash = hashlib.sha256()
    try:
        with open(ruta_archivo, "rb") as f:
            for byte_block in iter(lambda: f.read(4096), b""):
                sha256_hash.update(byte_block)
        return sha256_hash.hexdigest()
    except IOError:
        return None

def convertir_heic_a_jpg(ruta_heic, carpeta_path):
    """Convierte un archivo HEIC a JPG y mueve el original a la carpeta _convertidos."""
    filename = os.path.basename(ruta_heic)
    jpg_filename = os.path.splitext(filename)[0] + '.jpg'
    jpg_path = os.path.join(carpeta_path, jpg_filename)
    converted_path = os.path.join(carpeta_path, '_convertidos')
    os.makedirs(converted_path, exist_ok=True)

    if os.path.exists(jpg_path):
        print(f"  -> OMITIDO: El JPG '{jpg_filename}' ya existe. Moviendo HEIC original.")
    else:
        try:
            heif_file = pillow_heif.read_heif(ruta_heic)
            image = Image.frombytes(heif_file.mode, heif_file.size, heif_file.data, "raw")
            if image.mode in ("RGBA", "P"): image = image.convert("RGB")
            image.save(jpg_path, "JPEG")
            print(f"  -> CONVERTIDO: '{filename}' -> '{jpg_filename}'")
        except Exception as e:
            print(f"  -> FALLO DE CONVERSIÓN en '{filename}': {e}")
            return
    try:
        shutil.move(ruta_heic, converted_path)
    except Exception as e:
        print(f"  -> FALLO AL MOVER '{filename}' a '_convertidos': {e}")


def sanitizar_carpeta(carpeta_path):
    """Orquesta todo el proceso de limpieza para una carpeta."""
    print("\n" + "="*70)
    print(f"Sanitizando la carpeta: {os.path.basename(carpeta_path)}")
    print("="*70)

    if not os.path.exists(carpeta_path):
        print("La carpeta no existe. Saltando.")
        return

    # FASE 1: Conversión de HEICs
    print("--- Fase 1: Convirtiendo archivos HEIC a JPG ---")
    for filename in os.listdir(carpeta_path):
        ruta_completa = os.path.join(carpeta_path, filename)
        if os.path.isfile(ruta_completa) and filename.lower().endswith('.heic'):
            convertir_heic_a_jpg(ruta_completa, carpeta_path)

    # FASE 2: Detección de duplicados por contenido (Hash)
    print("\n--- Fase 2: Buscando duplicados reales por contenido (hash) ---")
    hashes = defaultdict(list)
    for filename in os.listdir(carpeta_path):
        ruta_completa = os.path.join(carpeta_path, filename)
        if os.path.isfile(ruta_completa) and not filename.startswith('_'):
            file_hash = calcular_hash(ruta_completa)
            if file_hash:
                hashes[file_hash].append(ruta_completa)
    
    duplicados_reales_movidos = 0
    for file_hash, files in hashes.items():
        if len(files) > 1:
            files.sort(key=lambda name: '(' in os.path.basename(name))
            for file_to_move in files[1:]:
                duplicados_path = os.path.join(carpeta_path, '_duplicados_reales')
                os.makedirs(duplicados_path, exist_ok=True)
                try:
                    shutil.move(file_to_move, duplicados_path)
                    duplicados_reales_movidos += 1
                    print(f"  -> DUPLICADO REAL: '{os.path.basename(file_to_move)}' movido a '_duplicados_reales'.")
                except Exception as e:
                     print(f"  -> FALLO AL MOVER DUPLICADO '{os.path.basename(file_to_move)}': {e}")

    # FASE 3: Detección y renombrado de colisiones de nombre
    print("\n--- Fase 3: Resolviendo colisiones de nombres (contenido único) ---")
    nombres_base = defaultdict(list)
    for filename in os.listdir(carpeta_path):
        if os.path.isfile(os.path.join(carpeta_path, filename)) and not filename.startswith('_'):
            match = re.match(r'(.+?)\s*\(\d+\)\.(.+)', filename)
            nombre_base = match.group(1) if match else os.path.splitext(filename)[0]
            nombres_base[nombre_base].append(filename)

    renombrados_automaticamente = 0
    for nombre_base, files in nombres_base.items():
        if len(files) > 1:
            print(f"  -> COLISIÓN DE NOMBRE detectada para '{nombre_base}'. Renombrando...")
            files.sort()
            for i, filename in enumerate(files):
                extension = os.path.splitext(filename)[1]
                nuevo_nombre = f"{nombre_base}_{i+1}{extension}"
                if filename != nuevo_nombre:
                     try:
                        os.rename(os.path.join(carpeta_path, filename), os.path.join(carpeta_path, nuevo_nombre))
                        renombrados_automaticamente += 1
                        print(f"     - '{filename}' -> '{nuevo_nombre}'")
                     except Exception as e:
                        print(f"     - FALLO AL RENOMBRAR '{filename}': {e}")

    print(f"\nResumen para '{os.path.basename(carpeta_path)}':")
    print(f" - Duplicados Reales (por contenido) movidos: {duplicados_reales_movidos}")
    print(f" - Colisiones de Nombre (contenido único) renombradas: {renombrados_automaticamente}")

# --- 2. Ejecutar la sanitización ---
if 'RUTA_BASE_PROYECTO' in locals():
    sanitizar_carpeta(RUTA_FOTOS_CRUDAS_PROPIAS)
    sanitizar_carpeta(RUTA_FOTOS_CRUDAS_EXTERNAS)
    print("\n\n✅ Proceso de sanitización completado.")
else:
    print("❌ ERROR: La variable 'RUTA_BASE_PROYECTO' no está definida. Ejecuta la celda de configuración (Paso 2) primero.")